In [1]:
%pylab inline
%matplotlib inline

# Global Imports
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import numpy as np
import pandas as pd
from sklearn import datasets
import os
import sys
import pickle
from pprint import pprint
from time import time
import datetime
from time import gmtime, strftime
import statsmodels.api as sm
from patsy import dmatrices

# Scikit-Learn imports
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import useful_methods

Populating the interactive namespace from numpy and matplotlib


# 0. Data

In [2]:
def ReadEmolexDf():
    # Read game_infos as df
    dfGameInfo = useful_methods.csv_dic_df(paths.READ_PATH_GAME_INFO + 'game_infos.csv')
    dfGameInfo = useful_methods.DropNanGames(dfGameInfo).copy().reset_index(drop=True)
    dfGameInfo.GW = [int(gw) for gw in dfGameInfo.GW]
    dfGameInfo = dfGameInfo.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)

    # Read Hash Emolex Model result
    df = useful_methods.csv_dic_df(paths.READ_PATH_RESULTS + RESULT_FILE_NAME)
    df.GW = [int(gw) for gw in df.GW]
    df = df.sort_values(['GW', 'away_team'], ascending=[True, True]).copy().reset_index(drop=True)

    df.emolex_home = [np.array([float(emo.strip()) for emo in emolex_home[1:-2].split('.')]) for emolex_home in list(df.emolex_home)]
    df.emolex_away = [np.array([float(emo.strip()) for emo in emolex_away[1:-2].split('.')]) for emolex_away in list(df.emolex_away)]


    # Combine 2 dfs
    df['score_ht_away'] = [int(item) for item in dfGameInfo.score_ht_away]
    df['score_ht_home'] = [int(item) for item in dfGameInfo.score_ht_home]
    df['score_ft_away'] = [int(item) for item in dfGameInfo.score_ft_away]
    df['score_ft_home'] = [int(item) for item in dfGameInfo.score_ft_home]
    
    df['GW'] = [int(item) for item in dfGameInfo.GW]

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    def Labeling(goal_diff):
        if goal_diff > 0:
            return 1
        elif goal_diff < 0:
            return 0
        else:
            return 2

    df['goal_diff_ht'] = df.score_ht_home - df.score_ht_away
    df['goal_diff_ft'] = df.score_ft_home - df.score_ft_away
    df['result'] = [Labeling(item) for item in df.goal_diff_ft] 

    return df

In [3]:
# Create df for models.
def CreateDfForModel(ht_draw=False, ft_wld=False):
    df = ReadEmolexDf()
        
    # only for Win or Lose
    if not ft_wld:
        df = df[df.result != 2].copy().reset_index(drop=True)
    
    # HT: Equal
    if ht_draw:
        df = df[df.goal_diff_ht == 0].copy().reset_index(drop=True)
    
    dta = pd.DataFrame()
    
    # Teams
    dta['team_home'] = df.home_team
    dta['team_away'] = df.away_team
    dta['GW'] = df.GW
    
    # HF scores
    dta['score_ht_home'] = df.score_ht_home
    dta['score_ht_away'] = df.score_ht_away
    
    dta['goal_diff_ht'] = df['goal_diff_ht']
    dta['goal_diff_ft'] = df['goal_diff_ft']

    # 'home_win': 1, 'away_win': 0, 'draw': 2
    dta['result'] = df.result
    
    # Emolex 8
    dta['anger_home'] = [emolex[0] / sum(emolex[:-2]) for emolex in df.emolex_home]
    dta['fear_home'] = [emolex[1] / sum(emolex[:-2]) for emolex in df.emolex_home]
    dta['disgust_home'] = [emolex[2] / sum(emolex[:-2]) for emolex in df.emolex_home]
    dta['sadness_home'] = [emolex[3] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['surprise_home'] = [emolex[4] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['trust_home'] = [emolex[5] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['joy_home'] = [emolex[6] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['anticipation_home'] = [emolex[7] / sum(emolex[:-2])  for emolex in df.emolex_home]
    dta['pos_home'] = [emolex[8] / sum(emolex[-2:])  for emolex in df.emolex_home]
    dta['neg_home'] = [emolex[9] / sum(emolex[-2:])  for emolex in df.emolex_home]

    dta['anger_away'] = [emolex[0] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['fear_away'] = [emolex[1] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['disgust_away'] = [emolex[2] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['sadness_away'] = [emolex[3] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['surprise_away'] = [emolex[4] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['trust_away'] = [emolex[5] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['joy_away'] = [emolex[6] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['anticipation_away'] = [emolex[7] / sum(emolex[:-2])  for emolex in df.emolex_away]
    dta['pos_away'] = [emolex[8] / sum(emolex[-2:])  for emolex in df.emolex_away]
    dta['neg_away'] = [emolex[9] / sum(emolex[-2:])  for emolex in df.emolex_away]
    
    
    # Diffs
    dta['diff_anger'] = dta['anger_home'] - dta['anger_away']
    dta['diff_fear'] = dta['fear_home'] - dta['fear_away']
    dta['diff_disgust'] = dta['disgust_home'] - dta['disgust_away']
    dta['diff_sadness'] = dta['sadness_home'] - dta['sadness_away']
    dta['diff_surprise'] = dta['surprise_home'] - dta['surprise_away']
    dta['diff_trust'] = dta['trust_home'] - dta['trust_away']
    dta['diff_joy'] = dta['joy_home'] - dta['joy_away']
    dta['diff_anticipation'] = dta['anticipation_home'] - dta['anticipation_away']
    dta['diff_pos'] = dta['pos_home'] - dta['pos_away']
    dta['diff_neg'] = dta['neg_home'] - dta['neg_away']
    
    
    return dta

# Multi-Class LOG

In [4]:
# Calculate Accuracy
# Log & SVM, cv = 18
def ModelAccuracy(X, y, penalty='l2', multi_class='ovr', log_variables=False):
    # Set Models
    model_log = LogisticRegression(penalty=penalty, multi_class=multi_class)

    # Fit to models
    model_log = model_log.fit(X, y)

    # Cross Validation
    scores_log = cross_val_score(model_log, X, y, scoring='accuracy', cv=13)

    # Accuracy scores
    print("--------------------------\n")
    print("[Log]: \t%.3f (cv: %.3f)" % (model_log.score(X, y), scores_log.mean()))


    # Variable Scores
    if log_variables:
        dfVars = pd.DataFrame()
        dfVars['var'] = X.columns
        dfVars['score'] = np.transpose(model_log.coef_)
        print("\n", dfVars)
    
    return model_log

## Grid Search

In [5]:
RESULT_FILE_NAME = "emolex_all_ht.csv"
df = CreateDfForModel(ft_wld=True)

# Results
# 1: Win, 2:Lose, 3:Draw
print(df["result"].value_counts())
print(df["result"].value_counts() / df['result'].count())

1    80
0    55
2    52
Name: result, dtype: int64
1    0.427807
0    0.294118
2    0.278075
Name: result, dtype: float64


In [11]:
# Create X, y
def CreateXy(df):
    # Create X, y for model
    y, X = dmatrices(
        'result ~ \
        anger_home + fear_home + disgust_home + sadness_home + \
        surprise_home + trust_home + joy_home + anticipation_home + \
        anger_away + fear_away + disgust_away + sadness_away + \
        surprise_away + trust_away + joy_away + anticipation_away + \
        pos_home + neg_home + pos_away + neg_away + \
        diff_anger + diff_fear + diff_disgust + diff_sadness + diff_surprise + \
        diff_trust + diff_joy + diff_anticipation + diff_pos + diff_neg',
        df, return_type="dataframe")
    y = np.ravel(y)

    return X, y

####################################

# Win Lose Dictionary
dfBin = df[(df.result != 2)].copy().reset_index(drop=True)
dfBin = df.copy()


# Test data is last 50 game
dfTrain = dfBin[dfBin.index < len(dfBin) - 50]
dfTest = dfBin[dfBin.index >= len(dfBin) - 50]

X_train, y_train = CreateXy(dfTrain)
X_test, y_test = CreateXy(dfTest)
X, y = CreateXy(dfBin)

In [7]:
# =============================================================
# =============================================================
# Train Model

# Pipelines
pipeline = Pipeline([
        ('clf', LogisticRegression(penalty='l2'))
    ])


# Hyper Parameters
parameters = {
    'clf__C': (0.1, 1, 10),
}


# Grid Search
grid_search = GridSearchCV(
    pipeline,
    parameters,
    n_jobs=-1,
    verbose=1,
    scoring='accuracy',
    cv=13
)

In [8]:
# =============================================================
# =============================================================

# Train Model
grid_search.fit(X_train, y_train)


print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')

best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
    
    

# =============================================================
# =============================================================

# Test Results
predictions = grid_search.predict(X_test)
print("------------------------------")
print("TEST:")
print('\n\nAccuracy:', metrics.accuracy_score(y_test, predictions))
print('\n\nConfusion Matrix:\n', metrics.confusion_matrix(y_test, predictions))
print('\n\nClassification Report:\n', metrics.classification_report(y_test, predictions))

Fitting 13 folds for each of 3 candidates, totalling 39 fits


[Parallel(n_jobs=-1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  33 out of  39 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:    0.1s finished


Best score: 0.753
Best parameters set:
	clf__C: 1
------------------------------
TEST:


Accuracy: 0.7


Confusion Matrix:
 [[ 9 10]
 [ 5 26]]


Classification Report:
              precision    recall  f1-score   support

        0.0       0.64      0.47      0.55        19
        1.0       0.72      0.84      0.78        31

avg / total       0.69      0.70      0.69        50



In [9]:
# Accuracy By Team
def TeamAccuracy(dfTest, team):
    # Team Accuracy
    dfTeam = dfTest[(dfTest.team_home == team) | (dfTest.team_away == team)].copy()
    results = dfTeam.result
    predictions = dfTeam.prediction

    print("----------------------------------")
    print(team)
    print('\n\nAccuracy:', metrics.accuracy_score(results, predictions))
    print('\n\nConfusion Matrix:\n', metrics.confusion_matrix(results, predictions))
    print('\n\nClassification Report:\n', metrics.classification_report(results, predictions))

    return team, metrics.accuracy_score(results, predictions)

In [10]:
# Test Games
predictions = grid_search.predict(X_test)
dfTest['prediction'] = predictions

teams = list(set(dfTest.team_home))
teams.sort()
[TeamAccuracy(dfTest, team) for team in teams]

----------------------------------
Arsenal


Accuracy: 0.6


Confusion Matrix:
 [[1 1]
 [1 2]]


Classification Report:
              precision    recall  f1-score   support

          0       0.50      0.50      0.50         2
          1       0.67      0.67      0.67         3

avg / total       0.60      0.60      0.60         5

----------------------------------
Bournemouth


Accuracy: 0.5


Confusion Matrix:
 [[0 2]
 [0 2]]


Classification Report:
              precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
          1       0.50      1.00      0.67         2

avg / total       0.25      0.50      0.33         4

----------------------------------
City


Accuracy: 0.75


Confusion Matrix:
 [[1 1]
 [0 2]]


Classification Report:
              precision    recall  f1-score   support

          0       1.00      0.50      0.67         2
          1       0.67      1.00      0.80         2

avg / total       0.83      0.75      0.73  

/Users/Bya/.virtualenvs/py3/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/Bya/.virtualenvs/py3/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[('Arsenal', 0.59999999999999998),
 ('Bournemouth', 0.5),
 ('City', 0.75),
 ('Crystal', 0.75),
 ('Everton', 1.0),
 ('Leicester', 0.5),
 ('Liverpool', 0.20000000000000001),
 ('Newcastle', 0.83333333333333337),
 ('Norwich', 0.8571428571428571),
 ('Southampton', 0.875),
 ('Stoke', 1.0),
 ('Sunderland', 0.40000000000000002),
 ('Swansea', 0.59999999999999998),
 ('Tottenham', 0.42857142857142855),
 ('United', 0.5),
 ('Villa', 1.0),
 ('Watford', 0.66666666666666663),
 ('WestBromwich', 1.0),
 ('WestHam', 0.59999999999999998)]

In [13]:
len([19, 19, 18, 19, 18, 19, 18, 19, 19, 19, 19, 19, 19, 19, 19, 18, 19, 17, 19, 19])

20

In [12]:
# Test Games
predictions = grid_search.predict(X)
df['prediction'] = predictions

teams = list(set(df.team_home))
teams.sort()
[TeamAccuracy(df, team) for team in teams]

----------------------------------
Arsenal


Accuracy: 0.578947368421


Confusion Matrix:
 [[2 4 0]
 [1 9 0]
 [1 2 0]]


Classification Report:
              precision    recall  f1-score   support

          0       0.50      0.33      0.40         6
          1       0.60      0.90      0.72        10
          2       0.00      0.00      0.00         3

avg / total       0.47      0.58      0.51        19

----------------------------------
Bournemouth


Accuracy: 0.578947368421


Confusion Matrix:
 [[3 2 0]
 [0 8 0]
 [2 4 0]]


Classification Report:
              precision    recall  f1-score   support

          0       0.60      0.60      0.60         5
          1       0.57      1.00      0.73         8
          2       0.00      0.00      0.00         6

avg / total       0.40      0.58      0.46        19

----------------------------------
Chelsea


Accuracy: 0.5


Confusion Matrix:
 [[3 2 0]
 [0 6 0]
 [3 4 0]]


Classification Report:
              precision    recall  f1

/Users/Bya/.virtualenvs/py3/lib/python3.4/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[('Arsenal', 0.57894736842105265),
 ('Bournemouth', 0.57894736842105265),
 ('Chelsea', 0.5),
 ('City', 0.63157894736842102),
 ('Crystal', 0.55555555555555558),
 ('Everton', 0.52631578947368418),
 ('Leicester', 0.3888888888888889),
 ('Liverpool', 0.31578947368421051),
 ('Newcastle', 0.68421052631578949),
 ('Norwich', 0.63157894736842102),
 ('Southampton', 0.63157894736842102),
 ('Stoke', 0.73684210526315785),
 ('Sunderland', 0.57894736842105265),
 ('Swansea', 0.47368421052631576),
 ('Tottenham', 0.42105263157894735),
 ('United', 0.3888888888888889),
 ('Villa', 0.63157894736842102),
 ('Watford', 0.6470588235294118),
 ('WestBromwich', 0.52631578947368418),
 ('WestHam', 0.47368421052631576)]